# Лабораторная работа № 8. Определение критически важных элементов модельной РЛС.

В работе предлагается провести свободный анализ функционирования модельной РЛС и выявить критически важные элементы, влияющие больше всего на излучаемую мощность РЛС.

## Описание данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv("gdrive/MyDrive/Учёба 4 курс 1 семестр/ ИУС и анализ больших данных/Lab8_data.csv")
df.head()

Mounted at /content/gdrive


,Timestamp,Regime,Power1,Power2,Power3,Power4,Power5,Power6,Power7,Power8,...,State10,State11,State12,State13,State14,State15,State16,Sum_power,State_float,State
0,0,3,0.0,355.82,410.17,402.41,330.94,0.0,368.30,0.0,...,1,1,0,0,0,1,1,3609.67,0.68,0
1,1,3,0.0,376.54,377.97,361.09,346.02,0.0,367.39,0.0,...,1,1,0,0,0,1,1,3599.12,0.68,0
2,2,2,0.0,345.88,396.03,354.06,347.23,0.0,380.34,0.0,...,1,1,0,0,0,0,1,3319.82,0.58,0
3,3,3,0.0,336.71,382.53,395.38,304.73,0.0,384.53,0.0,...,1,1,0,0,0,1,1,3637.85,0.67,0
4,4,1,0.0,369.41,382.03,376.33,339.97,0.0,391.72,0.0,...,1,1,0,0,0,1,1,3662.08,0.73,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 37 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Timestamp    10000 non-null  int64  
 1   Regime       10000 non-null  int64  
 2   Power1       10000 non-null  float64
 3   Power2       10000 non-null  float64
 4   Power3       10000 non-null  float64
 5   Power4       10000 non-null  float64
 6   Power5       10000 non-null  float64
 7   Power6       10000 non-null  float64
 8   Power7       10000 non-null  float64
 9   Power8       10000 non-null  float64
 10  Power9       10000 non-null  float64
 11  Power10      10000 non-null  float64
 12  Power11      10000 non-null  float64
 13  Power12      10000 non-null  float64
 14  Power13      10000 non-null  float64
 15  Power14      10000 non-null  float64
 16  Power15      10000 non-null  float64
 17  Power16      10000 non-null  float64
 18  State1       10000 non-null  int64  
 19  State

Файл с данными предствляет собой запись мощности, излучаемой модельной РЛС, состоящей из 16 блоков усиления мощности (БУМ). Динамика мощности отражает реальные процессы и взаимосвязи, происходящие на РЛС.

Первый столбец обозначает временной отсчет, второй столбец - режим работы РЛС, который напрямую определяет выходную мощность БУМ и сказывается на внутренних процессах деградации БУМ. Далее в 16 столбцах записаны мощности выходного сигнала БУМ, а также состояния каждого из блоков усиления мощности, которое показывает, дает ли этот блок вклад в излучаемую мощность РЛС. Следом представлена суммарная мощность, излучаемая РЛС, а также состояние РЛС в виде вещественного числа, которое показывает, насколько излучаемая мощность соответствует максимально возможной при данном режиме работы РЛС. Последний столбец - информация о состоянии РЛС в виде бинарной величины.

Каждый БУМ характеризуется своим состоянием, который показывает, насколько выдаваемая мощность соответствует максимально возможной для этого БУМ. Снижение работоспособности БУМ (снижение его состояния) связано исключительно с работой самого БУМ: чем дольше и напряженнее он работает, тем быстрее снижается его состояние. Кроме того, блоки немного отличаются друг от друга, т.е. каждый из них снижает свою рабоспособность немного с разной скоростью.

В каждом из режимов работы есть диапазон возможных излучаемых мощностей. В самом простом режиме работы (зондирование местности) изменение состояния БУМ не происходит. В самом напряженном режиме работы (сопровождение одновременно нескольких целей) уменьшение состояния БУМ происходит быстрее всего.

При снижении выдаваемой мощности ниже определенной отметки, оператор принимает решение о замене текущего БУМ новым, и блок перестает работать - его выходная мощность равна 0. Если таких блоков накопится несколько, то они заменяеются на абсолютно новые, состояние которых равно 1.

Иногда мощность блоков независимо от действующего режима опускается ниже 300 Вт (случайное резкое снижение мощности) на 1 отсчет. После этого блоки продолжают работать также, как и ранее.

Каждый БУМ имеет свой вклад в суммарную мощность РЛС. Суммарная излучаемая мощность РЛС равна сумме выдаваемой мощности каждого БУМ, умноженного на коэффициент влияния БУМ (вещественное число от 0 до 1). Под отказом РЛС (значение 0 в последнем столбце) понимается снижение суммарной мощности на 10 % от максимально возможной при заданном режиме работы.

## Задания

### 1. Определите диапазон выходных мощностей БУМ для каждого из режимов работы. Определите среднюю мощность и стандартное отклонение. Изобразите эту информацию визуально.

In [4]:
# Определение диапазона выходных мощностей БУМ для каждого режима работы
power_ranges = {}
for regime in df["Regime"].unique():
    regime_data = df[df["Regime"] == regime]
    power_ranges[regime] = {
        f"Power{i}": (regime_data[f"Power{i}"].min(), regime_data[f"Power{i}"].max())
        for i in range(1, 17)
    }

# Вывод диапазонов выходных мощностей
for regime, ranges in power_ranges.items():
    print(f"Regime: {regime}")
    for power, power_range in ranges.items():
        print(f"{power}: {power_range}")
    print()


Regime: 3
Power1: (0.0, 439.49)
Power2: (0.0, 437.05)
Power3: (0.0, 439.12)
Power4: (0.0, 439.54)
Power5: (0.0, 437.31)
Power6: (0.0, 438.55)
Power7: (0.0, 436.53)
Power8: (0.0, 439.42)
Power9: (0.0, 437.88)
Power10: (0.0, 437.22)
Power11: (0.0, 439.78)
Power12: (0.0, 440.52)
Power13: (0.0, 439.18)
Power14: (0.0, 436.53)
Power15: (0.0, 439.85)
Power16: (0.0, 439.23)

Regime: 2
Power1: (0.0, 425.83)
Power2: (0.0, 426.68)
Power3: (0.0, 424.13)
Power4: (0.0, 426.52)
Power5: (0.0, 426.51)
Power6: (0.0, 427.14)
Power7: (0.0, 425.57)
Power8: (0.0, 426.42)
Power9: (0.0, 426.46)
Power10: (0.0, 425.49)
Power11: (0.0, 427.25)
Power12: (0.0, 427.35)
Power13: (0.0, 426.7)
Power14: (0.0, 425.69)
Power15: (0.0, 426.48)
Power16: (0.0, 427.45)

Regime: 1
Power1: (0.0, 404.54)
Power2: (0.0, 404.29)
Power3: (0.0, 404.68)
Power4: (0.0, 404.99)
Power5: (0.0, 404.41)
Power6: (0.0, 404.36)
Power7: (0.0, 404.73)
Power8: (0.0, 404.81)
Power9: (0.0, 404.97)
Power10: (0.0, 404.71)
Power11: (0.0, 404.85)
Power12

In [ ]:
# Расчет средней мощности и стандартного отклонения
average_power = df.iloc[:, 2:18].mean()
std_dev_power = df.iloc[:, 2:18].std()

# Визуализация
plt.figure(figsize=(10, 6))

# График средней мощности (линией)
plt.subplot(2, 1, 1)
plt.plot(average_power.index, average_power, marker='o', linestyle='-')
plt.title('Average Power of BUMs')
plt.xlabel('BUMs')
plt.ylabel('Average Power')

# График стандартного отклонения (линией)
plt.subplot(2, 1, 2)
plt.plot(std_dev_power.index, std_dev_power, marker='o', linestyle='-')
plt.title('Standard Deviation of BUMs Power')
plt.xlabel('BUMs')
plt.ylabel('Standard Deviation')

plt.tight_layout()
plt.show()

### 2. Определите для каждого БУМ его подверженность деградации при работе по назначению. Изобразите эту информацию визуально.

In [ ]:
# Расчет усредненного состояния для каждого БУМа в различных режимах работы
average_states = {}
for bum in range(1, 17):
    bum_states = {}
    for regime in df["Regime"].unique():
        regime_data = df[df["Regime"] == regime]
        bum_states[regime] = regime_data[f"State{bum}"].mean()
    average_states[f"BUM_{bum}"] = bum_states

# Визуализация
plt.figure(figsize=(12, 8))

for bum, states in average_states.items():
    plt.plot(list(states.keys()), list(states.values()), marker='o', label=bum)

plt.title('Average State of BUMs in Different Regimes')
plt.xlabel('Regimes')
plt.ylabel('Average State')
plt.legend()
plt.show()

### 3. Определите вклад каждого БУМ в излучаемую мощность РЛС. Изобразите эту информацию визуально. Выделите критически важные элементы - блоки, которые в большей степени влияют на излучаемую мощность РЛС.

In [ ]:
# Визуализация вклада каждого БУМа в излучаемую мощность РЛС
bum_columns = [f"Power{i}" for i in range(1, 17)]

plt.figure(figsize=(12, 8))

# Расчет нормализованной мощности каждого БУМа
total_power = df[bum_columns].sum(axis=1)
normalized_powers = df[bum_columns].div(total_power, axis=0)

# Расчет вклада каждого БУМа
total_power_contributions = []
for bum_column in bum_columns:
    contribution = df[bum_column] * normalized_powers[bum_column]
    total_power_contributions.append(contribution)

# Добавление столбца суммарного вклада
df["TotalPowerContribution"] = sum(total_power_contributions)

# Визуализация вклада каждого БУМа
for bum_column in bum_columns:
    plt.plot(df["Timestamp"], df[bum_column] * normalized_powers[bum_column], label=bum_column)

# Визуализация суммарного вклада
plt.plot(df["Timestamp"], df["TotalPowerContribution"], label="Total Power Contribution", linewidth=2, linestyle="--")

plt.title('Contribution of Each BUM to Radar Power (Normalized)')
plt.xlabel('Timestamp')
plt.ylabel('Power Contribution')
plt.legend()
plt.show()

# Анализ критически важных элементов
critical_bums = df[bum_columns].sum().idxmax()  # Находим БУМ с наибольшим суммарным вкладом
print(f"The most critical BUM in terms of total power contribution: {critical_bums}")

### 4. Определите, каково среднее время жизни каждого БУМ. Изобразите эту информацию визуально.

In [1]:
def ttl(pow):

    time = []
    start = 0
    end = 0
    count_started = False

    for i in range(len(pow)):
        if (pow[i]!=0 and count_started==False):
            start = i
            count_started = True

        if(i>0):
            if (pow[i]==pow[i-1]==0 and count_started):
                end = i
                count_started = False
                time.append(end - start)

    return np.mean(time)

for i in range(16):
    print('TTL for Block '+str(i+1)+' '+str(ttl(df['Power'+str(i+1)])))

NameError: name 'df' is not defined

### 5. Сделайте выводы.